In [1]:
from keras.utils import multi_gpu_model
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

C:\python3_27\envs\tensorflow-gpu\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\python3_27\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\python3_27\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\python3_27\envs\tensorflow-gpu\lib\site-p

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 747207812235079101
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4943878553
locality {
  bus_id: 1
  links {
  }
}
incarnation: 440094980914651047
physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [2]:
import os

import numpy as np
import json
import codecs
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf

from keras.layers import Input, Lambda, add
from keras.models import Model
from keras.optimizers import Adam, SGD
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, LearningRateScheduler
from keras.backend.tensorflow_backend import set_session 

from yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss
from yolo3.utils import get_random_data

os.environ["CUDA_VISIBLE_DEVICES"]="0"

def _main():
    
    annotation_path = 'data/2007_train.txt'
    log_dir = 'logs/'
    classes_path = 'model_data/voc_classes.txt'
    anchors_path = 'model_data/yolo_anchors.txt'
    class_names = get_classes(classes_path)
    num_classes = len(class_names)
    anchors = get_anchors(anchors_path)

    input_shape = (416,416) # multiple of 32, hw
    is_tiny_version = len(anchors)==6 # default setting
    # use tiny model if need else darkent53 model for train
    if is_tiny_version:
        
         pass
        
        #with tf.device('/cpu:0'):
            #template_model = create_tiny_model(input_shape, anchors, num_classes,
                #freeze_body=1, weights_path='model_data/yolo_weights.h5')
            #print('model.input shape', model.input_layers)
    else:
        '''build the darknet53 model on the cpu'''
        with tf.device('/cpu:0'):
            template_model = create_model(input_shape, anchors, num_classes,
                freeze_body=1, weights_path='model_data/yolo.h5') # make sure you know what you freeze
#*******************************************************************            
            #model_data/yolo.h5
            #logs/200張200次yolo沒動植/ep225-loss18.263-val_loss17.069.h5
            #logs/200張200次yolo沒動植/wurenji.h5
     
    
    
    
    
    
    
    logging = TensorBoard(log_dir=log_dir)
    checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
                                 monitor='val_loss', save_weights_only=True, save_best_only=True, period=3)
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)

    # template_model input
    # print('model inputs', model.input)
    '''add shedule for reduce lr ratio by epoch monitor'''
    def shedule(epoch):
        if epoch <= 150:
            return 1e-3
        elif epoch <= 170:
            return 1e-4
        else:
            return 1e-5
    
    lr_shedule = LearningRateScheduler(shedule)
    
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

    val_split = 0.15
    with open(annotation_path) as f:
        lines = f.readlines()
    np.random.seed(10101)
    np.random.shuffle(lines)
    np.random.seed(None)
    num_val = int(len(lines)*val_split)
    num_train = len(lines) - num_val

    '''
    write test code for multi_gpus
    '''
    model = template_model
    print('use the multi_gpu_model for model training ')

    # for layer in template_model.layers:
    #     layer.trainable = True
    # print(model.summary())

    # Train with frozen layers first, to get a stable loss.
    # Adjust num epochs to your dataset. This step is enough to obtain a not bad model.
    if True:
        #print('model inputlayer', template_model.input_layers)
        optimizer =Adam(lr=0.01)
        #optimizer =Adam(lr=0.01)
        #optimizer =SGD(lr=0.001)
        #optimizer =SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
        model.compile(optimizer=optimizer, loss={
            # use custom yolo_loss Lambda layer.
            'yolo_loss': lambda y_true, y_pred: y_pred})
        print("the gpu-model compile is ok!!")
        batch_size = 16
        
        # model = multi_gpu_model(model, gpus=2)
        print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
        history_logs = model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
                steps_per_epoch=max(1, num_train//batch_size),
                validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
                validation_steps=max(1, num_val//batch_size),
                epochs=500,
                initial_epoch=0,
                # callbacks=[logging, checkpoint, lr_shedule]
                callbacks=[logging, lr_shedule, checkpoint])
        # wirte loss record to a json file
        with codecs.open('logs/jiaotong_1/history_logs.json', 'w', 'utf-8') as outfile:
            json.dump(history_logs.history, outfile, ensure_ascii=False)
            outfile.write('\n')
 
        '''template_model summary'''
        # print(template_model.summary())
        '''save the model config and weigths on cpu field share the weights'''
        # with tf.device('/cpu:0'):
        template_model.save(log_dir+'wurenji.h5')

    '''loads_wights of trained_weights_stage1.h5'''
    
'''functions'''
def get_loss_fig():
    epochs = 40
    data = []
    with codecs.open("logs/113/history_logs.json", "r", "utf-8") as f:
        for line in f:
            dic = json.loads(line)
            data.append(dic)
    logs = data[0]
    loss = logs['loss']
    val_loss = logs['val_loss']
    x = [i+1 for i in range(epochs)]
    plt.figure(0)
    plt.plot(x, loss, 'r-', label='loss')
    plt.plot(x, val_loss, 'y-', label='val_loss')
    plt.legend(loc='best')
    plt.show()
    plt.savefig('traffic_loss.png')

def get_classes(classes_path):
    '''loads the classes'''
    with open(classes_path, encoding='utf8') as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)

#******************************************************
def create_model(input_shape, anchors, num_classes, load_pretrained=True, freeze_body=2,
            weights_path='model_data/yolo.h5'):
    '''create the training model'''
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)

    y_true = [Input(shape=(h//{0:32, 1:16, 2:8}[l], w//{0:32, 1:16, 2:8}[l], \
        num_anchors//3, num_classes+5)) for l in range(3)]

    model_body = yolo_body(image_input, num_anchors//3, num_classes)
    print('Create YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze darknet53 body or freeze all but 3 output layers.
            num = (185, len(model_body.layers)-3)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))
        else:
            for i in range(len(model_body.layers)):
                model_body.layers[i].trainable = True
            print('UnFreeze all of the model_body layers.')

    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.5})([*model_body.output, *y_true])
        
        #([*model_body.output + y_true])
    model = Model([model_body.input, *y_true], model_loss)

    return model



def data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes):
    '''data generator for fit_generator'''
    n = len(annotation_lines)
    i = 0
    while True:
        image_data = []
        box_data = []
        for b in range(batch_size):
            if i==0:
                np.random.shuffle(annotation_lines)
            image, box = get_random_data(annotation_lines[i], input_shape, random=True)
            image_data.append(image)
            box_data.append(box)
            i = (i+1) % n
        image_data = np.array(image_data)
        box_data = np.array(box_data)
        y_true = preprocess_true_boxes(box_data, input_shape, anchors, num_classes)
        yield [image_data, *y_true], np.zeros(batch_size)

def data_generator_wrapper(annotation_lines, batch_size, input_shape, anchors, num_classes):
    n = len(annotation_lines)
    if n==0 or batch_size<=0: return None
    return data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes)

def _get_available_devices():
    return [x.name for x in K.get_session().list_devices()]


def _normalize_device_name(name):
    name = '/' + ':'.join(name.lower().replace('/', '').split(':')[-2:])
    return name

'''modify the devices('/cpu:0') concatenate to add for yolo loss tensor(1,)'''


def multi_gpu_model(model, gpus=None):
    
    if K.backend() != 'tensorflow':
        raise ValueError('`multi_gpu_model` is only available '
                         'with the TensorFlow backend.')

    available_devices = _get_available_devices()
    available_devices = [_normalize_device_name(name) for name in available_devices]
    if not gpus:
        # Using all visible GPUs when not specifying `gpus`
        # e.g. CUDA_VISIBLE_DEVICES=0,2 python3 keras_mgpu.py
        gpus = len([x for x in available_devices if 'gpu' in x])

    if isinstance(gpus, (list, tuple)):
        if len(gpus) <= 1:
            raise ValueError('For multi-gpu usage to be effective, '
                             'call `multi_gpu_model` with `len(gpus) >= 2`. '
                             'Received: `gpus=%s`' % gpus)
        num_gpus = len(gpus)
        target_gpu_ids = gpus
    else:
        if gpus <= 1:
            raise ValueError('For multi-gpu usage to be effective, '
                             'call `multi_gpu_model` with `gpus >= 2`. '
                             'Received: `gpus=%d`' % gpus)
        num_gpus = gpus
        target_gpu_ids = range(num_gpus)

    import tensorflow as tf

    target_devices = ['/cpu:0'] + ['/gpu:%d' % i for i in target_gpu_ids]
    for device in target_devices:
        if device not in available_devices:
            raise ValueError(
                'To call `multi_gpu_model` with `gpus=%d`, '
                'we expect the following devices to be available: %s. '
                'However this machine only has: %s. '
                'Try reducing `gpus`.' % (gpus,
                                          target_devices,
                                          available_devices))

    def get_slice(data, i, parts):
        shape = tf.shape(data)
        batch_size = shape[:1]
        input_shape = shape[1:]
        step = batch_size // parts
        if i == num_gpus - 1:
            size = batch_size - step * i
        else:
            size = step
        size = tf.concat([size, input_shape], axis=0)
        stride = tf.concat([step, input_shape * 0], axis=0)
        start = stride * i
        return tf.slice(data, start, size)

    all_outputs = []
    for i in range(len(model.outputs)):
        all_outputs.append([])

    # Place a copy of the model on each GPU,
    # each getting a slice of the inputs.
    for i, gpu_id in enumerate(target_gpu_ids):
        with tf.device('/gpu:%d' % gpu_id):
            with tf.name_scope('replica_%d' % gpu_id):
                inputs = []
                # Retrieve a slice of the input.
                for x in model.inputs:
                    input_shape = tuple(x.get_shape().as_list())[1:]
                    slice_i = Lambda(get_slice,
                                     output_shape=input_shape,
                                     arguments={'i': i,
                                                'parts': num_gpus})(x)
                    inputs.append(slice_i)

                # Apply model on slice
                # (creating a model replica on the target device).
                outputs = model(inputs)
                if not isinstance(outputs, list):
                    outputs = [outputs]

                # Save the outputs for merging back together later.
                for o in range(len(outputs)):
                    all_outputs[o].append(outputs[o])

    # Merge outputs on CPU.
    with tf.device('/cpu:0'):
        merged = []
        for name, outputs in zip(model.output_names, all_outputs):
            merged.append(add(outputs, name=name))
        return Model(model.inputs, merged)


if __name__ == '__main__':
    
    # memory of gpu allowcate
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    config.log_device_placement=True
    sess = tf.Session(config=config)
    set_session(sess)
    
    
    _main()

Create YOLOv3 model with 9 anchors and 1 classes.


C:\python3_27\envs\tensorflow-gpu\lib\site-packages\keras\engine\saving.py:1009: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((1, 1, 1024, 18) vs (255, 1024, 1, 1)).
  weight_values[i].shape))
C:\python3_27\envs\tensorflow-gpu\lib\site-packages\keras\engine\saving.py:1009: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((18,) vs (255,)).
  weight_values[i].shape))
C:\python3_27\envs\tensorflow-gpu\lib\site-packages\keras\engine\saving.py:1009: UserWarning: Skipping loading of weights for layer conv2d_67 due to mismatch in shape ((1, 1, 512, 18) vs (255, 512, 1, 1)).
  weight_values[i].shape))
C:\python3_27\envs\tensorflow-gpu\lib\site-packages\keras\engine\saving.py:1009: UserWarning: Skipping loading of weights for layer conv2d_67 due to mismatch in shape ((18,) vs (255,)).
  weight_values[i].shape))
C:\python3_27\envs\tensorflow-gpu\lib\site-packages\keras\engine\saving.py:1009: UserWarning: Skipping loa

Load weights model_data/yolo.h5.
Freeze the first 185 layers of total 252 layers.
use the multi_gpu_model for model training 
the gpu-model compile is ok!!
Train on 924 samples, val on 162 samples, with batch size 16.
Epoch 1/500
57/57 [==============================] - 87s 2s/step - loss: 25.1893 - val_loss: 15.7237
Epoch 2/500
57/57 [==============================] - 72s 1s/step - loss: 14.6581 - val_loss: 14.4497
Epoch 3/500
57/57 [==============================] - 69s 1s/step - loss: 13.7881 - val_loss: 13.9215
Epoch 4/500
57/57 [==============================] - 68s 1s/step - loss: 13.2801 - val_loss: 13.3514
Epoch 5/500
57/57 [==============================] - 69s 1s/step - loss: 12.9616 - val_loss: 13.0587
Epoch 6/500
57/57 [==============================] - 68s 1s/step - loss: 12.7678 - val_loss: 12.5050
Epoch 7/500
57/57 [==============================] - 66s 1s/step - loss: 12.5386 - val_loss: 12.2219
Epoch 8/500
57/57 [==============================] - 66s 1s/step - loss: 12

Epoch 159/500
57/57 [==============================] - 66s 1s/step - loss: 10.0895 - val_loss: 10.1755
Epoch 160/500
57/57 [==============================] - 65s 1s/step - loss: 10.1961 - val_loss: 10.2354
Epoch 161/500
57/57 [==============================] - 66s 1s/step - loss: 10.2596 - val_loss: 10.1888
Epoch 162/500
57/57 [==============================] - 65s 1s/step - loss: 10.1567 - val_loss: 10.5763
Epoch 163/500
57/57 [==============================] - 65s 1s/step - loss: 10.0260 - val_loss: 10.3055
Epoch 164/500
57/57 [==============================] - 65s 1s/step - loss: 10.1194 - val_loss: 10.3086
Epoch 165/500
57/57 [==============================] - 65s 1s/step - loss: 10.1455 - val_loss: 10.1143
Epoch 166/500
57/57 [==============================] - 66s 1s/step - loss: 10.0760 - val_loss: 10.4354
Epoch 167/500
57/57 [==============================] - 64s 1s/step - loss: 10.1515 - val_loss: 10.1655
Epoch 168/500
57/57 [==============================] - 66s 1s/step - loss

57/57 [==============================] - 64s 1s/step - loss: 10.0359 - val_loss: 10.2127
Epoch 319/500
57/57 [==============================] - 66s 1s/step - loss: 10.0276 - val_loss: 10.0988
Epoch 320/500
57/57 [==============================] - 65s 1s/step - loss: 9.9367 - val_loss: 10.5476
Epoch 321/500
57/57 [==============================] - 65s 1s/step - loss: 9.9381 - val_loss: 10.1104
Epoch 322/500
57/57 [==============================] - 65s 1s/step - loss: 10.1029 - val_loss: 10.1875
Epoch 323/500
57/57 [==============================] - 64s 1s/step - loss: 10.0088 - val_loss: 10.3318
Epoch 324/500
57/57 [==============================] - 66s 1s/step - loss: 10.0022 - val_loss: 10.1352
Epoch 325/500
57/57 [==============================] - 65s 1s/step - loss: 9.8839 - val_loss: 10.2944
Epoch 326/500
57/57 [==============================] - 66s 1s/step - loss: 10.1381 - val_loss: 10.1948
Epoch 327/500
57/57 [==============================] - 64s 1s/step - loss: 9.9569 - val_lo

57/57 [==============================] - 66s 1s/step - loss: 10.0009 - val_loss: 10.1742
Epoch 399/500
57/57 [==============================] - 65s 1s/step - loss: 10.0227 - val_loss: 10.0995
Epoch 400/500
57/57 [==============================] - 65s 1s/step - loss: 9.9887 - val_loss: 10.1149
Epoch 401/500
57/57 [==============================] - 65s 1s/step - loss: 9.9483 - val_loss: 10.1977
Epoch 402/500
57/57 [==============================] - 65s 1s/step - loss: 9.9004 - val_loss: 10.2717
Epoch 403/500
57/57 [==============================] - 65s 1s/step - loss: 9.9925 - val_loss: 10.0804
Epoch 404/500
57/57 [==============================] - 64s 1s/step - loss: 10.0583 - val_loss: 9.9823
Epoch 405/500
57/57 [==============================] - 66s 1s/step - loss: 9.9881 - val_loss: 10.0312
Epoch 406/500
57/57 [==============================] - 64s 1s/step - loss: 9.9110 - val_loss: 10.1065
Epoch 407/500
57/57 [==============================] - 66s 1s/step - loss: 10.0582 - val_loss: